<a href="https://colab.research.google.com/github/kumardesappan/colab-notebooks/blob/main/save_huggingface_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install timm
!pip3 install onnx-simplifier

from transformers import DetrFeatureExtractor, DetrForObjectDetection
from PIL import Image
import requests
import torch
import torch.onnx

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = DetrFeatureExtractor.from_pretrained('facebook/detr-resnet-50')
model = DetrForObjectDetection.from_pretrained('facebook/detr-resnet-50')

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)

# model predicts bounding boxes and corresponding COCO classes
logits = outputs.logits
bboxes = outputs.pred_boxes

print(logits, bboxes)

name = 'detr_resnet-50.onnx'
inputs['pixel_mask'].shape
dummy_input = torch.ones(1, 3, 800, 1066, dtype=torch.float)
torch.onnx.export(model, dummy_input, name, verbose=True,opset_version=11)
!python3 -m onnxsim 'detr_resnet-50.onnx' 'detr_resnet-50-simplified.onnx'

In [ ]:
from transformers import SegformerFeatureExtractor, SegformerModel
feature_extractor_1 = SegformerFeatureExtractor.from_pretrained("nvidia/mit-b0")
model_1 = SegformerModel.from_pretrained("nvidia/mit-b0")
inputs_1 = feature_extractor_1(image, return_tensors="pt")
outputs_1 = model_1(**inputs_1)
print(inputs_1['pixel_values'].shape)
dummy_input_1 = torch.ones(1, 3, 512, 512, dtype=torch.float)
torch.onnx.export(model_1, dummy_input_1, 'segformer.onnx', verbose=True,opset_version=11)
!python3 -m onnxsim 'segformer.onnx' 'segformer-simplified.onnx'


torch.Size([1, 3, 512, 512])


/usr/local/lib/python3.7/dist-packages/torch/onnx/symbolic_helper.py:325: UserWarning: Type cannot be inferred, which might cause exported graph to produce incorrect results.
  warnings.warn("Type cannot be inferred, which might cause exported graph to produce incorrect results.")


graph(%input.1 : Float(1, 3, 512, 512, strides=[786432, 262144, 512, 1], requires_grad=0, device=cpu),
      %encoder.patch_embeddings.0.proj.weight : Float(32, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=1, device=cpu),
      %encoder.patch_embeddings.0.proj.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %encoder.patch_embeddings.0.layer_norm.weight : Float(32, strides=[1], requires_grad=1, device=cpu),
      %encoder.patch_embeddings.0.layer_norm.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %encoder.patch_embeddings.1.proj.weight : Float(64, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=1, device=cpu),
      %encoder.patch_embeddings.1.proj.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %encoder.patch_embeddings.1.layer_norm.weight : Float(64, strides=[1], requires_grad=1, device=cpu),
      %encoder.patch_embeddings.1.layer_norm.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %encoder.patch_embeddings